In [3]:
import os
#check if dataset/ exists
if not os.path.exists('dataset'):
    os.makedirs('dataset')
if not os.path.exists('dataset/train'):
    os.makedirs('dataset/train')
if not os.path.exists('dataset/test'):
    os.makedirs('dataset/test')

In [4]:
import pandas as pd
import numpy as np

#read the csv file
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')

In [5]:
from tqdm import tqdm
def make_dataset(df):
    pbar = tqdm(total=len(df['building_id'].unique()))
    if 'anomaly' in df.columns:
        path="train"
    else:
        path="test"
    for i in df['building_id'].unique():
        dataset=df[df['building_id'] == i].reset_index(drop=True)
        dataset.set_index('timestamp', inplace=True)
        dataset.sort_index(inplace=True)
        dataset["filled"] = dataset["meter_reading"].apply(lambda x: 1 if np.isnan(x) else 0)
        #linear interpolation on the meter_reading 

        dataset['meter_reading'] = dataset['meter_reading'].interpolate(method='linear')
        #apply on meter_reading truncate the values to 3 decimal places
        dataset['meter_reading'] = dataset['meter_reading'].apply(lambda x: np.round(x, 3))
        if path == "train":
            dataset['anomaly'] = dataset['anomaly'].fillna(0)
            dataset['anomaly'] = dataset['anomaly'].astype(int)
        dataset.dropna(inplace=True)
        dataset.to_csv('dataset/{}/{}.csv'.format(path,i), index=True)
        pbar.update(1)
    pbar.close()

In [6]:
make_dataset(train)
make_dataset(test)

100%|██████████| 206/206 [00:25<00:00,  8.00it/s]
